 # Peer-Graded Assignment : Segmenting and Clustering Neighborhoods in Toronto Part 1 
 - Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto. 
 - Get the geographical coordinates of the neighborhoods in Toronto.
 - Explore and cluster the neighborhoods in Toronto (replicate the same analysis we did to New York City data).
 ***
###  Import libraries 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries imported.")

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Dell\Anaconda3

  added / updated specs:
    - geopy


The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.3-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


## Scrap data from Wikipedia page into a DataFrame

In [8]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [9]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [10]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [15]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip(' ')) # avoid new lines in neighborhood cell

In [16]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


## Drop cells with a borough that is "Not assigned"

In [17]:
#drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned\n"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront
3,M6A\n,North York\n,Lawrence Manor / Lawrence Heights
4,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government


In [18]:
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge\n"
1,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G\n,Scarborough\n,"Woburn, Woburn\n"
4,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae\n"


## Clean up the syntax

In [21]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
    

toronto_df_grouped = toronto_df_grouped.replace('\n',' ', regex=True)
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern / Rouge, Malvern / Rouge"
1,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E,Scarborough,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


## Print the number of rows of the cleaned dataframe

In [24]:
toronto_df_grouped.shape

(103, 3)